In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.func import *
from model_based_mvpa.models.model import *

In [2]:
root = '/data2/project_model_based_fmri/ds001882/'

In [3]:
def prep_func(row,info):

    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    if row['delay_left'] >= row['delay_right']:
        new_df['delay_later'] = row['delay_left']
        new_df['delay_sooner'] = row['delay_right']
        new_df['amount_later'] = row['money_left']
        new_df['amount_sooner'] = row['money_right']
        new_df['choice'] = 1 if row['choice'] == 1 else 0
    else:
        new_df['delay_later'] = row['delay_right']
        new_df['delay_sooner'] = row['delay_left']
        new_df['amount_later'] = row['money_right']
        new_df['amount_sooner'] = row['money_left']
        new_df['choice'] = 1 if row['choice'] == 2 else 0
        
    return new_df

In [4]:
def cond_func(df_row):
    return df_row['agent']==0

In [5]:
def latent_func(row,info,k,beta):
    
    new_df = row.copy()
    new_df['subjID'] = info['subject']
    new_df['run'] = info['run']
    
    if row['delay_left'] >= row['delay_right']:
        ev_later   = row['money_left'] / (1 + k * row['delay_left'])
        ev_sooner  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    else:
        ev_sooner   = row['money_left'] / (1 + k * row['delay_left'])
        ev_later  = row['money_right'] / (1 + k * row['delay_right'])
        utility = ev_later - ev_sooner
    new_df['modulation'] = utility
    
    return new_df

In [6]:
par_names = ['k','beta']

In [7]:
dm_model = 'dd_hyperbolic'

In [9]:
dm_model, df_events, signals, time_masks = preprocess_events(root, dm_model,   
                                                              latent_func,par_names,
                                                              prep_func,
                                                              cond_func,
                                                              df_events=None,
                                                              all_ind_pars=None,
                                                              use_duration=False,
                                                              layout=None,
                                                              hrf_model='glover',
                                                              save_path='.',
                                                              save=True,
                                                              ncore=os.cpu_count(),
                                                              time_check=True)

      hbayesdm doing (model: dd_hyperbolic)..      :  50%|█████     | 3/6 [00:29<00:36, 12.32s/it]INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using cached StanModel: cached-dd_hyperbolic-pystan_2.19.1.1.pkl



Model  = dd_hyperbolic
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 88
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 20
 # of (max) trials per subject  = 120

Using cached StanModel: cached-dd_hyperbolic-pystan_2.19.1.1.pkl


        calculating modulation..        :  67%|██████▋   | 4/6 [02:41<01:36, 48.27s/it]           

Index(['onset', 'duration', 'choice', 'agent', 'moneyleft', 'delayleft',
       'moneyright', 'delayright', 'subjid', 'run', 'delaylater',
       'delaysooner', 'amountlater', 'amountsooner'],
      dtype='object')
['onset', 'duration', 'choice', 'agent', 'money_left', 'delay_left', 'money_right', 'delay_right', 'subjID', 'run', 'delay_later', 'delay_sooner', 'amount_later', 'amount_sooner']
************************************
**** Model fitting is complete! ****
************************************


       modulation signal making..       :  83%|████████▎ | 5/6 [02:43<00:34, 34.35s/it]/home/cheoljun/anaconda3/envs/model_based_fmri/lib/python3.7/site-packages/model_based_mvpa-0.0.1-py3.7.egg/model_based_mvpa/preprocessing/events.py:195: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  signals = np.array(signals)
       events preproecssing done!       : 100%|██████████| 6/6 [02:44<00:00, 27.35s/it]
